In [1]:
import cutax
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

/project2/lgrandi/xenonnt/software/cutax/v1.19.5/cutax/cut_lists/science.py:28: UserWarning: Removing cut <class 'cutax.cuts.s2_width.S2Width'>
  warnings.warn(f"Removing cut {cut}", UserWarning)
/project2/lgrandi/xenonnt/software/cutax/v1.19.5/cutax/cut_lists/science.py:28: UserWarning: Removing cut <class 'cutax.cuts.s2_width.S2Width'>
  warnings.warn(f"Removing cut {cut}", UserWarning)


In [2]:
output_folder = "/scratch/midway2/yuem/data_availibility"

In [3]:
st = cutax.contexts.xenonnt_offline(xedocs_version="global_v16",output_folder=output_folder)

You specified _auto_append_rucio_local=True and you are not on dali compute nodes, so we will add the following rucio local path: /project/lgrandi/rucio/


/opt/XENONnT/anaconda/envs/XENONnT_el7.sr1_wimp_unblind/lib/python3.9/site-packages/straxen/config/preprocessors.py:16: UserWarning: From straxen version 2.1.0 onward, URLConfig parameterswill be sorted alphabetically before being passed to the plugins, this will change the lineage hash for non-sorted URLs. To load data processed with non-sorted URLs, you will need to use an older version.
  warnings.warn(
Unknown config option xedocs_version; will do nothing.
Invalid context option xedocs_version; will do nothing.


In [15]:
# SR1a background runs
sr1a_bkg_runs = st.select_runs(
        run_mode=['*background*','tpc_bkg'],
        exclude_tags=['bad', 'messy', 'flash'],
        include_tags=['*sr1a*']
    )
    
# SR1b background runs
sr1b_bkg_runs = st.select_runs(
        run_mode=['*background*','tpc_bkg'],
        exclude_tags=['bad', 'messy', 'flash'],
        include_tags=['*sr1b*']
    )
    
# SR1 Rn220 runs 
sr1_rn220_runs = st.select_runs(
        run_mode=['tpc_radon','tpc_radon_hev'],
        exclude_tags=['bad', 'messy', 'flash'],
        include_tags=['*sr1b*']
    )
    
# SR1 Rn222 runs 
sr1_rn222_runs = st.select_runs(
        run_mode=['tpc_radon222'],
        exclude_tags=['bad', 'messy', 'flash'],
        include_tags=['*sr1b*']
    )

Checking data availability:   0%|          | 0/2 [00:00<?, ?it/s]

Checking data availability:   0%|          | 0/2 [00:00<?, ?it/s]

Checking data availability:   0%|          | 0/2 [00:00<?, ?it/s]

Checking data availability:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
sr1a_kr_runs = st.select_runs(
        run_mode='tpc_kr83m',
        exclude_tags=['bad', 'messy', 'flash'],
        include_tags=['*sr1a*']
    )

sr1b_kr_runs = st.select_runs(
        run_mode='tpc_kr83m',
        exclude_tags=['bad', 'messy', 'flash'],
        include_tags=['*sr1b*']
    )

Checking data availability:   0%|          | 0/2 [00:00<?, ?it/s]

Checking data availability:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
sr1_ambe_runs = st.select_runs(
        run_mode='*ambe*',
        exclude_tags=['bad', 'messy', 'flash'],
        include_tags=['*sr1*']
    )

Checking data availability:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
from tqdm import tqdm 

def add_is_stored_info(runlist):
    """
    Add is_stored info to the runlist
    """
    runlist.reset_index(drop=True, inplace=True)
    # Create the new columns
    runlist.loc[:, 'is_stored_event_basics'] = np.full(len(runlist), False)
    runlist.loc[:, 'is_stored_event_area_per_channel'] = np.full(len(runlist), False)
    
    # Update values using .loc
    for i, run in tqdm(enumerate(runlist.name.values)):
        if st.is_stored(run, 'event_basics'):
            runlist.loc[i, 'is_stored_event_basics'] = True

        if st.is_stored(run, 'event_area_per_channel'):
            runlist.loc[i, 'is_stored_event_area_per_channel'] = True
            
def add_load_availibility_info(runlist):
    """
    Add load_availibility info to the runlist
    """
    runlist.reset_index(drop=True, inplace=True)
    # Create the new columns
    runlist.loc[:, 'load_event_basics'] = np.full(len(runlist), False)
    runlist.loc[:, 'load_event_area_per_channel'] = np.full(len(runlist), False)
    
    # Update values using .loc
    for i, run in tqdm(enumerate(runlist.name.values)):
        if st.is_stored(run, 'event_basics'):
            try:
                st.get_array(run, 'event_basics',progress_bar=False)
                runlist.loc[i, 'load_event_basics'] = True
            except Exception as e:
                print(f"Error loading event_basics for run {run}: {e}")
                runlist.loc[i, 'load_event_basics'] = False

        if st.is_stored(run, 'event_area_per_channel'):
            try:
                st.get_array(run, 'event_area_per_channel',progress_bar=False)
                runlist.loc[i, 'load_event_area_per_channel'] = True
            except Exception as e:
                print(f"Error loading event_area_per_channel for run {run}: {e}")
                runlist.loc[i, 'load_event_area_per_channel'] = False

### Rn220

In [27]:
len(sr1_rn220_runs)

179

In [29]:
add_is_stored_info(sr1_rn220_runs)

179it [00:07, 25.08it/s]


In [37]:
add_load_availibility_info(sr1_rn220_runs)

179it [08:43,  2.92s/it]


In [50]:
sr1_rn220_runs[~sr1_rn220_runs.load_event_area_per_channel].name.values

array(['049841', '049701', '049655', '049610', '049544', '049505',
       '049450', '049430'], dtype=object)

### Rn222

In [42]:
add_is_stored_info(sr1_rn222_runs)
add_load_availibility_info(sr1_rn222_runs)

379it [00:20, 18.69it/s]
379it [05:18,  1.19it/s]


In [43]:
sr1_rn222_runs.is_stored_event_basics.value_counts()

True     351
False     28
Name: is_stored_event_basics, dtype: int64

In [44]:
sr1_rn222_runs.is_stored_event_area_per_channel.value_counts()

True     351
False     28
Name: is_stored_event_area_per_channel, dtype: int64

In [52]:
sr1_rn222_runs[~sr1_rn222_runs.load_event_basics].name.values

array(['051602', '051341', '051329', '051278', '051266', '051241',
       '051215', '051159', '051105', '051089', '051080', '051074',
       '051066', '051064', '051063', '051062', '051056', '051052',
       '051043', '051039', '051032', '051028', '051020', '051017',
       '051004', '050995', '050982', '050972'], dtype=object)

In [53]:
sr1_rn222_runs[~sr1_rn222_runs.load_event_area_per_channel].name.values

array(['051602', '051341', '051329', '051278', '051266', '051241',
       '051215', '051159', '051105', '051089', '051080', '051074',
       '051066', '051064', '051063', '051062', '051056', '051052',
       '051043', '051039', '051032', '051028', '051020', '051017',
       '051004', '050995', '050982', '050972'], dtype=object)

### AmBe

In [11]:
add_is_stored_info(sr1_ambe_runs)
add_load_availibility_info(sr1_ambe_runs)

167it [00:06, 25.06it/s]
167it [04:38,  1.66s/it]


In [12]:
sr1_ambe_runs[~sr1_ambe_runs.load_event_basics].name.values

array(['051942', '051929', '051926', '051925', '051918', '051912',
       '051698', '051380'], dtype=object)

In [13]:
sr1_ambe_runs[~sr1_ambe_runs.load_event_area_per_channel].name.values

array(['051942', '051929', '051926', '051925', '051918', '051912',
       '051698', '051380'], dtype=object)

### SR1a Kr83m

In [7]:
add_is_stored_info(sr1a_kr_runs)
add_load_availibility_info(sr1a_kr_runs)

222it [00:08, 25.23it/s]
222it [10:56,  2.96s/it]


In [8]:
sr1a_kr_runs[~sr1a_kr_runs.load_event_basics].name.values

array(['048657', '048653', '048648', '048637', '048256', '048234',
       '047612', '047596', '047595', '047200', '046793', '046784',
       '046778', '043286'], dtype=object)

In [9]:
sr1a_kr_runs[~sr1a_kr_runs.load_event_area_per_channel].name.values

array(['048657', '048653', '048648', '048642', '048637', '048262',
       '048259', '048256', '048253', '048234', '047612', '047596',
       '047595', '047593', '047221', '047206', '047200', '046802',
       '046793', '046787', '046784', '046781', '046778', '043301',
       '043286', '043271'], dtype=object)

### SR1a BKG

In [16]:
add_is_stored_info(sr1a_bkg_runs)
add_load_availibility_info(sr1a_bkg_runs)

1763it [00:57, 30.86it/s]
1763it [27:54,  1.05it/s]


In [18]:
sr1a_bkg_runs[~sr1a_bkg_runs.load_event_basics].name.values

array(['048546', '048523', '048522', '048521', '048514', '048513',
       '048507', '048490', '048489', '048488', '048487', '048486',
       '048485', '048477', '048476', '048475', '048474', '048473',
       '048468', '048467', '048466', '048465', '048464', '048463',
       '048461', '048460', '048459', '048458', '048457', '048456',
       '048455', '048454', '048452', '048451', '047621', '044256'],
      dtype=object)

In [19]:
sr1a_bkg_runs[~sr1a_bkg_runs.load_event_area_per_channel].name.values

array(['048546', '048523', '048522', '048521', '048514', '048513',
       '048507', '048490', '048489', '048488', '048487', '048486',
       '048485', '048477', '048476', '048475', '048474', '048473',
       '048468', '048467', '048466', '048465', '048464', '048463',
       '048461', '048460', '048459', '048458', '048457', '048456',
       '048455', '048454', '048452', '048451', '047621', '044256'],
      dtype=object)

### SR1b BKG

In [20]:
add_is_stored_info(sr1b_bkg_runs)
add_load_availibility_info(sr1b_bkg_runs)

0it [00:00, ?it/s]

783it [00:24, 32.53it/s]
783it [32:16,  2.47s/it]


In [25]:
sr1b_bkg_runs[~sr1b_bkg_runs.load_event_basics].name.values

array(['053505', '053504', '053503'], dtype=object)

In [24]:
sr1b_bkg_runs[~sr1b_bkg_runs.load_event_area_per_channel].name.values

array(['053505', '053504', '053503', '053502'], dtype=object)